# Thêm Drive vào colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cài đặt thư viện cần thiết

In [2]:
!pip install PyMuPDF chromadb sentence-transformers tqdm regex
!pip install pdfplumber
!pip install fuzzywuzzy python-Levenshtein unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 21.2 MB/s et

# Tải model lên Drive (để gọi lại sau này)

In [ ]:
from sentence_transformers import SentenceTransformer
import os

# Đường dẫn nơi bạn muốn lưu mô hình trên Google Drive
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(DRIVE_MODEL_PATH, exist_ok=True)
print(f"Thư mục lưu mô hình: {DRIVE_MODEL_PATH}")

In [ ]:
from sentence_transformers import SentenceTransformer
EMB_MODEL_NAME = "BAAI/bge-m3"

# Khởi tạo mô hình (đã tải hoặc đang tải nếu chưa có trong cache)
model = SentenceTransformer(EMB_MODEL_NAME)

# Lưu mô hình vào Drive
# Đây là bước quan trọng nhất
model.save(DRIVE_MODEL_PATH)

print("✅ Đã lưu mô hình thành công vào Google Drive.")

# Tạo lại Collection (khi chuyển tài khoản)

In [1]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import chromadb.api

# === 1️⃣ Cấu hình (Đã sửa lỗi đường dẫn) ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store" # Đường dẫn chuẩn
COLLECTION = "so_tay_hcmue"
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'
INPUT_TXT = "/content/drive/MyDrive/chatbotHCMUE/content/chunks.txt" # Tệp nguồn

# === 2️⃣ Đọc Dữ liệu Nguồn ===
try:
    with open(INPUT_TXT, "r", encoding="utf-8") as f:
        docs = [x.strip() for x in f.read().split("\n\n") if len(x.strip()) > 80]
    print(f"📘 Tổng {len(docs)} đoạn nội dung đã được tải từ {INPUT_TXT}.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file nguồn '{INPUT_TXT}'.")
    docs = []

if not docs:
    print("⚠️ Không có dữ liệu để tạo Collection. Vui lòng kiểm tra lại file nguồn.")
else:
    # === 3️⃣ Thêm Metadata (Bắt buộc) ===
    # Tạo danh sách metadata: [ {"source": "So_Tay_Chinh"}, {"source": "So_Tay_Chinh"}, ... ]
    metadatas = [{"source": "So_Tay_Chinh"} for _ in docs]

    # === 4️⃣ RESET và TẠO LẠI Collection ===
    chromadb.api.client.SharedSystemClient._instance = None
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings(allow_reset=True))

    try:
        client.delete_collection(COLLECTION)
    except:
        pass

    col = client.create_collection(COLLECTION)
    print(f"✅ Đã tạo mới Collection '{COLLECTION}'.")

    # === 5️⃣ Tạo Embeddings và Thêm vào Collection ===
    try:
        model = SentenceTransformer(DRIVE_MODEL_PATH)
        print("🚀 Đang tạo embeddings...")

        embs = []
        for i in tqdm(range(0, len(docs), 32)):
            batch = docs[i:i+32]
            batch_emb = model.encode(batch, normalize_embeddings=True).tolist()
            embs.extend(batch_emb)

        # Thêm dữ liệu vào Collection (BẮT BUỘC PHẢI CÓ THAM SỐ METADATAS)
        col.add(
            ids=[str(i) for i in range(len(docs))],
            documents=docs,
            embeddings=embs,
            metadatas=metadatas # <-- ĐÃ THÊM THAM SỐ NÀY
        )

        final_count = col.count()
        print(f"\n✅ Đã lưu {final_count} đoạn vào collection '{COLLECTION}'.")
        print("Collection đã được tạo lại thành công và sẵn sàng để sử dụng.")

    except Exception as e:
        print(f"\n❌ Lỗi trong quá trình tạo embeddings/add dữ liệu: {e}")

📘 Tổng 411 đoạn nội dung đã được tải từ /content/drive/MyDrive/chatbotHCMUE/content/chunks.txt.
✅ Đã tạo mới Collection 'so_tay_hcmue'.
🚀 Đang tạo embeddings...


100%|██████████| 13/13 [00:35<00:00,  2.75s/it]



✅ Đã lưu 411 đoạn vào collection 'so_tay_hcmue'.
Collection đã được tạo lại thành công và sẵn sàng để sử dụng.


# Tải lại Collection

In [3]:
import chromadb
from chromadb.config import Settings
import os

DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"

if not os.path.isdir("/content/drive"):
    print("⚠️ Lỗi: Google Drive chưa được Mount.")
elif not os.path.isdir(DB_DIR):
    print(f"⚠️ Lỗi: Không tìm thấy thư mục DB tại: {DB_DIR}")
else:
    # === 2️⃣ Tải lại Collection ===
    try:
        # 1. Khởi tạo client, nó sẽ tự động load dữ liệu từ DB_DIR
        client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())

        # 2. KIỂM TRA và Khắc phục lỗi "count 0"
        # Thử get_or_create: Nếu collection đã tồn tại, nó sẽ lấy về;
        # Nếu không, nó sẽ tạo một cái mới (nhưng trong trường hợp của bạn, nó sẽ đọc từ các file đã có)
        col = client.get_or_create_collection(COLLECTION)

        # 3. Kiểm tra lại:
        current_count = col.count()
        print(f"\n✅ Đã tải thành công Collection '{COLLECTION}' từ Drive.")
        print(f"Tổng số tài liệu trong Collection: {current_count}")

        if current_count == 0:
            print("⚠️ LỖI VẪN TIẾP DIỄN: Dữ liệu DB có thể bị hỏng hoặc lỗi quyền truy cập.")
            print("Vui lòng kiểm tra lại cấu trúc thư mục 'vector_store' trong Drive mới.")

    except Exception as e:
        print(f"\n❌ Lỗi: {e}")


✅ Đã tải thành công Collection 'so_tay_hcmue' từ Drive.
Tổng số tài liệu trong Collection: 411


#Kiểm tra trạng thái Collection (metadata và số lượng)

In [4]:
import chromadb
from chromadb.config import Settings
import os

# === SỬA LỖI ĐƯỜNG DẪN DRIVE CHUẨN ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"
MIN_EXPECTED_COUNT = 400 # Số lượng tài liệu Sổ tay tối thiểu bạn mong đợi

try:
    # 1. Tải Collection
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
    col = client.get_collection(COLLECTION)

    # 2. Kiểm tra tổng số tài liệu (Chỉ dùng count() không có tham số)
    total_count = col.count()
    print(f"✅ Tổng số tài liệu trong Collection: {total_count}")

    # 3. KIỂM TRA DỮ LIỆU SỔ TAY BẰNG col.get()
    # Lấy ID của các tài liệu có source là So_Tay_Chinh, sau đó đếm số lượng
    # Ta chỉ lấy IDs để tối ưu tốc độ
    so_tay_results = col.get(
        where={"source": {"$eq": "So_Tay_Chinh"}},
        include=[] # Chỉ lấy ID để tối ưu tốc độ
    )
    so_tay_count = len(so_tay_results['ids'])
    print(f"✅ Số tài liệu thuộc nguồn Sổ Tay (So_Tay_Chinh): {so_tay_count}")

    # 4. Đưa ra kết luận
    if so_tay_count == 0:
        print("\n❌ LỖI NẶNG: Collection SỔ TAY bị rỗng.")
        print("   Giải pháp: CHẠY LẠI code tạo Collection Sổ Tay (chỉ chunks.txt).")
    elif so_tay_count < MIN_EXPECTED_COUNT:
        print(f"\n⚠️ LỖI: Dữ liệu Sổ Tay bị thiếu (Cần {MIN_EXPECTED_COUNT}, có {so_tay_count}).")
        print("   Giải pháp: CHẠY LẠI code tạo Collection Sổ Tay.")
    else:
        print("\n🎉 Dữ liệu Sổ Tay đầy đủ và sẵn sàng để truy vấn Vector.")

except Exception as e:
    print(f"\n❌ LỖI NẶNG: Không thể tải Collection. {e}")

✅ Tổng số tài liệu trong Collection: 411
✅ Số tài liệu thuộc nguồn Sổ Tay (So_Tay_Chinh): 411

🎉 Dữ liệu Sổ Tay đầy đủ và sẵn sàng để truy vấn Vector.


# Truy vấn dữ liệu

In [2]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import re, json, pandas as pd
from fuzzywuzzy import fuzz
from unidecode import unidecode
import chromadb.api
from tqdm import tqdm
import os
import sys
from typing import List, Dict, Anyc

# === 0️⃣ HÀM HỖ TRỢ ===

def remove_vietnamese_diacritics(text: str) -> str:
    """Sử dụng unidecode để loại bỏ dấu tiếng Việt và chuyển sang chữ thường."""
    return unidecode(text).lower()

def classify_query_intent(query: str) -> str:
    """Xác định xem truy vấn là về Môn học hay Sổ tay/Chung."""

    # 1. Chuẩn hóa câu hỏi nhập vào
    normalized_query = remove_vietnamese_diacritics(query)

    # 2. Định nghĩa các từ khóa và cụm từ khóa môn học đã chuẩn hóa

    # DANH SÁCH TỪ KHÓA CHUYÊN NGÀNH ĐƯỢC MỞ RỘNG (Phrase Matching)
    course_phrases = [
        "lap trinh co ban", "co so toan", "toan roi rac", "thiet ke web",
        "duong loi quoc phong", "phap luat dai cuong", "triet hoc mac lenin",
        "tam ly hoc", "giao duc the chat", "lap trinh nang cao",
        "lap trinh huong doi tuong", "cong tac quoc phong", "kinh te chinh tri",
        "chu nghia xa hoi", "phuong phap nghien cuu khoa hoc", "giao duc doi song",
        "phuong phap hoc tap", "ky nang thich ung", "ky nang lam viec nhom",
        "cau truc du lieu", "co so du lieu", "lap trinh tren windows",
        "xac suat thong ke", "ly thuyet do thi", "quan su chung",
        "tu tuong ho chi minh", "kien truc may tinh", "nhap mon mang may tinh",
        "he dieu hanh", "phan tich va thiet ke giai thuat", "quy hoach tuyen tinh",
        "ky thuat chien dau", "lich su dang cong san", "nhap mon cong nghe phan mem",
        "phan tich thiet ke huong doi tuong", "tri tue nhan tao", "cac he co so du lieu",
        "thiet ke va quan ly mang lan", "phan tich va thiet ke he thong thong tin",
        "co so du lieu nang cao", "he thong ma nguon mo", "xu ly anh so",
        "quan tri co ban voi windows server", "nghi thuc giao tiep mang",
        "phat trien ung dung tren thiet bi di dong", "quan ly du an cong nghe thong tin",
        "kiem thu phan mem", "phat trien ung dung tro choi",
        "quy trinh phat trien phan mem agile", "he thong nhung", "hoc may",
        "lap trinh python", "lap trinh php", "thuc hanh nghe nghiep",
        "mang may tinh nang cao", "cong nghe web", "cong nghe java",
        "cac he co so tri thuc", "do hoa may tinh", "bao mat va an ninh mang",
        "logic mo", "cong nghe net", "chuyen de oracle", "truyen thong ky thuat so",
        "chuan doan va quan ly su co mang", "dinh tuyen mang nang cao",
        "quan tri mang voi linux", "quan tri dich vu mang",
        "he thong quan tri doanh nghiep", "xay dung du an cong nghe thong tin",
        "he tu van thong tin", "bao mat co so du lieu", "khai thac du lieu va ung dung",
        "lap trinh tien hoa", "cac phuong phap hoc thong ke",
        "lap rap cai dat va bao tri may tinh", "internet van vat", "nhap mon devops",
        "cong nghe chuoi khoi", "cac giai thuat tinh toan dai so",
        "khai thac du lieu van ban", "xu ly ngon ngu tu nhien", "ly thuyet ma hoa va mat ma",
        "thuc tap nghe nghiep", "khoi nghiep", "cong nghe phan mem nang cao",
        "cong nghe mang khong day", "thuong mai dien tu", "kiem thu phan mem nang cao",
        "dien toan dam may", "do hoa may tinh nang cao", "phan tich du lieu",
        "may hoc nang cao", "thi giac may tinh", "phan tich anh y khoa",
        "phat trien ung dung tren thiet bi di dong nang cao", "khoa luan tot nghiep",
        "ho so tot nghiep", "san pham nghien cuu",
        # Thêm các từ khóa mô tả ý định
        "mo ta mon", "thong tin mon", "hoc phan", "mon hoc", "hoc gi"
    ]

    # DANH SÁCH TỪ KHÓA ĐƠN MẠNH (Single-Token Keywords)
    strong_single_keywords = [
        "mon", "hoc phan", "lap trinh", "toan", "python", "java", "web",
        "du lieu", "ai", "linux", "windows", "thong ke", "do hoa", "bao mat",
        "mang", "phap luat", "triet hoc", "tam ly", "lich su", "kinh te",
        "phat trien", "thiet ke", "quantri", "server", "oracle", "devops",
        "agile", "khoi nghiep", "vat ly", "hoa hoc", "su pham", "tin hoc",
        "huong doi tuong"
    ]


    # 3. KIỂM TRA 1: Khớp cụm từ hoặc toàn bộ tên môn học
    if any(phrase in normalized_query for phrase in course_phrases):
        return "COURSE"

    # 4. KIỂM TRA 2: Khớp từ khóa đơn cho truy vấn ngắn (đảm bảo nó là tên môn học)
    if len(normalized_query.split()) <= 4 and any(k in normalized_query for k in strong_single_keywords):
        return "COURSE"

    return "GENERAL" # Mọi thứ khác là truy vấn Sổ tay/Chung

def pretty(text: str) -> str:
    """Định dạng văn bản để hiển thị dễ đọc hơn."""
    text = re.sub(r"([;:.\)\]\}])\s*(?=[\+\-•–])", r"\1\n", text)
    text = re.sub(r"\s*([+\-•–])\s+", r"\n\1 ", text)
    text = re.sub(r"\s*(Điều\s+\d+\.)", r"\n\1", text, flags=re.IGNORECASE)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


# === 1️⃣ CẤU HÌNH & ĐƯỜNG DẪN ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'
COLLECTION = "so_tay_hcmue"
TABLE_JSON = "/content/drive/MyDrive/chatbotHCMUE/so_tay_all_tables_clean.json"
COURSE_JSON = "/content/drive/MyDrive/chatbotHCMUE/mon_hoc_mo_ta.json"


# === 2️⃣ HÀM TRA BẢNG JSON (Khớp mờ) ===
def find_table_by_keyword(query: str) -> str | None:
    """Tìm kiếm bảng phù hợp nhất trong TABLE_JSON dựa trên khớp mờ."""

    if 'tables' not in globals() or not tables:
        return None

    normalized_query = remove_vietnamese_diacritics(query)

    mapping = {
        "4 sang chữ": ["thang_diem_4"],
        "học bổng": ["xep_loai_hoc_bong"],
        "chữ sang 10": ["thang_diem_10_chu"],
        "thang điểm 10 sang chữ": ["thang_diem_10_chu"],
        "thang điểm 4 sang chữ": ["thang_diem_4"],
        "xếp loại học lực": ["xep_loai_hoc_luc"],
        "xếp loại học bổng": ["xep_loai_hoc_bong"],
        "yêu cầu học bổng": ["yeu_cau_hoc_bong"],
        "điểm rèn luyện": ["diem_ren_luyen1", "diem_ren_luyen2", "diem_ren_luyen3", "diem_ren_luyen4", "diem_ren_luyen5"],
    }

    BEST_MATCH_SCORE = 90
    best_match_key = None
    best_score = 0
    final_result = ""

    # 1. KIỂM TRA KHỚP HOÀN HẢO
    for map_key in mapping.keys():
        if normalized_query == remove_vietnamese_diacritics(map_key):
            best_match_key = map_key
            best_score = 100
            break

    # 2. KHỚP MỜ (FUZZY MATCHING)
    if not best_match_key:
        for map_key in mapping.keys():
            normalized_map_key = remove_vietnamese_diacritics(map_key)
            score_wratio = fuzz.WRatio(normalized_query, normalized_map_key)
            score_partial = fuzz.partial_ratio(normalized_query, normalized_map_key)
            score = max(score_wratio, score_partial)

            if score > best_score and score >= BEST_MATCH_SCORE:
                best_score = score
                best_match_key = map_key

    # 3. Xử lý và THU THẬP TẤT CẢ các bảng liên quan
    if best_match_key:
        types_to_export = mapping[best_match_key]

        final_result += f"### 📊 Kết quả Tra cứu Bảng (Độ khớp: {best_score}%)\n"

        for type_name in types_to_export:
            for t in tables:
                if t["type"] == type_name:
                    df = pd.DataFrame(t["data"])
                    table_title = t.get("title", type_name.replace("_", " ").title())
                    final_result += f"\n#### Bảng: {table_title}\n"
                    final_result += df.to_markdown(index=False)
                    final_result += "\n"

    if final_result:
        return final_result
    else:
        return None

# === 3️⃣ HÀM KHỚP MỜ MÔN HỌC (Tối ưu) ===
def find_course_by_fuzzy_match(query: str) -> Dict[str, Any] | None:
    """Sử dụng Khớp Mờ (Token Set Ratio) để tìm tên môn học chính xác nhất."""

    global COURSE_DATA
    if not COURSE_DATA:
        return None

    normalized_query = remove_vietnamese_diacritics(query)
    BEST_SCORE = 90 # Ngưỡng tin cậy Khớp Mờ

    best_match_key = None
    best_score = 0

    for course_key, data in COURSE_DATA.items():
        # Sử dụng Token Set Ratio: Bỏ qua thứ tự và thiếu từ
        score = fuzz.token_set_ratio(normalized_query, course_key)

        # Dùng WRatio/Partial Ratio làm backup nếu Token Set không đạt yêu cầu cao
        score = max(score, fuzz.WRatio(normalized_query, course_key))
        score = max(score, fuzz.partial_ratio(normalized_query, course_key))

        if score > best_score and score >= BEST_SCORE:
            best_score = score
            best_match_key = course_key

    if best_match_key:
        course_data = COURSE_DATA[best_match_key]
        # Trả về cấu trúc kết quả để dễ dàng hiển thị
        return {
            "documents": [[f"Môn học: {course_data['Ten_Mon']}. Mô tả: {course_data['Description']}"]],
            "metadatas": [[{
                "source": "Mon_Hoc_JSON_FUZZY",
                "Ten_Mon": course_data['Ten_Mon'],
                "Score": best_score
            }]]
        }
    return None

# === 4️⃣ KHỞI TẠO HỆ THỐNG VÀ BẮT ĐẦU CHAT ===

# Reset instance tránh lỗi "already exists"
chromadb.api.client.SharedSystemClient._instance = None
client, col, model, tables = None, None, None, [] # Khởi tạo biến
global COURSE_DATA
COURSE_DATA = {} # Biến toàn cục để lưu dữ liệu môn học


try:
    # 4.1 Load Client và Collection (Chỉ chứa Sổ tay)
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
    col = client.get_collection(COLLECTION)

    # 4.2 Load Mô hình từ Drive
    model = SentenceTransformer(DRIVE_MODEL_PATH)

    # 4.3 Load bảng JSON (tables)
    with open(TABLE_JSON, "r", encoding="utf-8") as f:
        tables = json.load(f)

    # 4.4 LOAD DỮ LIỆU MÔN HỌC TỪ JSON (COURSE_DATA)
    with open(COURSE_JSON, "r", encoding="utf-8") as f:
        course_list = json.load(f)

    for course_item in course_list:
        key = remove_vietnamese_diacritics(course_item['ten_mon'])
        COURSE_DATA[key] = {
            "Ten_Mon": course_item['ten_mon'],
            "Description": course_item['Description']
        }

    print("✅ Hệ thống đã sẵn sàng.")
    print(f"   Tổng tài liệu Sổ tay (Vector): {col.count()}")
    print(f"   Tổng môn học (Fuzzy): {len(COURSE_DATA)}")
except Exception as e:
    print(f"❌ Lỗi khởi tạo hệ thống: {e}. Vui lòng kiểm tra lại trạng thái Drive/Mô hình/Collection.")
    sys.exit(1)


# === 5️⃣ Giao diện chat CLI ===

if all([client, col, model]): # Chỉ chạy nếu khởi tạo thành công
    print("\n=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE ===")
    print("Nhập 'exit' để thoát.\n")

    while True:
        q = input("📘 Nhập nội dung muốn tra cứu: ").strip()
        if not q:
            continue
        if q.lower() == "exit":
            print("👋 Tạm biệt!")
            break

        # Kiểm tra độ dài
        if len(remove_vietnamese_diacritics(q)) < 5:
            print("⚠️ Vui lòng nhập chi tiết hơn.")
            print("\n-----------------------------\n")
            continue

        # --- 5.1 Kiểm tra có bảng phù hợp không (Tra cứu bằng Fuzzy) ---
        table_result = find_table_by_keyword(q)
        if table_result:
            print("\n🔎 Kết quả tìm thấy (BẢNG):\n")
            print(table_result)
            print("\n-----------------------------\n")
            continue

        # --- 5.2 Kiểm tra có MÔN HỌC phù hợp không (Tra cứu bằng Khớp Mờ) ---
        intent = classify_query_intent(q)
        res = None

        if intent == "COURSE":
            # 1. THỬ KHỚP MỜ MÔN HỌC TRỰC TIẾP (Độ chính xác cao)
            res = find_course_by_fuzzy_match(q)

            if res:
                print("\n[Hệ thống: Phân loại MÔN HỌC. Đã tìm thấy 1 kết quả bằng Khớp Mờ.]")
            else:
                # 2. KHÔNG TÌM THẤY BẰNG KHỚP MỜ, CHUYỂN SANG TÌM KIẾM CHUNG (GENERAL)
                intent = "GENERAL"
                print("\n[Hệ thống: Phân loại MÔN HỌC thất bại. Chuyển sang tìm kiếm SỔ TAY.]")

        # --- 5.3 Truy vấn Sổ tay (Dùng Vector Search) nếu intent là GENERAL ---
        if intent == "GENERAL":
            query_kwargs = {"include": ['documents', 'metadatas'], "n_results": 5}
            # CHỈ TÌM TRONG SỔ TAY (Loại trừ sự nhiễu của môn học)
            query_kwargs["where"] = {"source": {"$eq": "So_Tay_Chinh"}}

            try:
                q_emb = model.encode(q, normalize_embeddings=True).tolist()
                res = col.query(query_embeddings=[q_emb], **query_kwargs)
                print("\n[Hệ thống: Phân loại SỔ TAY. Đã tìm thấy 5 kết quả bằng Vector Search.]")
            except Exception as e:
                print(f"\n❌ Lỗi Encoding/Truy vấn: {e}.")
                res = None

        # KIỂM TRA KẾT QUẢ TRUY VẤN
        if not res or not res["documents"] or not res["documents"][0] or not res["documents"][0][0].strip():
            print("\n❌ Không tìm thấy thông tin liên quan trong Sổ tay Sinh viên.")
            print("Vui lòng thử từ khóa khác hoặc chi tiết hơn.")
            print("\n-----------------------------\n")
            continue

        # --- 5.4 In kết quả và phân biệt nguồn ---
        print("\n🔎 Kết quả tìm thấy:")
        for i, doc in enumerate(res["documents"][0]):
            metadata = res["metadatas"][0][i]

            if doc.strip():
                source_type = metadata.get('source', 'Sổ tay')

                if source_type == 'Mon_Hoc_JSON_FUZZY':
                    # Dữ liệu từ Khớp Mờ Môn học
                    ten_mon = metadata.get('Ten_Mon', 'Tên Môn học không rõ')
                    score = metadata.get('Score', 'N/A')
                    print(f"\n========================================================")
                    print(f"📚 KẾT QUẢ TỐT NHẤT (MÔN HỌC - KHỚP MỜ): {ten_mon}")
                    print(f"   Nguồn: {source_type} (Độ khớp: {score}%)")
                    print(pretty(doc))
                    print(f"========================================================")

                else:
                    # Dữ liệu văn bản Sổ tay
                    print(f"\n📄 Đoạn {i+1} (Nguồn: {source_type})")
                    print(pretty(doc))

        print("\n-----------------------------\n")

✅ Hệ thống đã sẵn sàng.
   Tổng tài liệu Sổ tay (Vector): 411
   Tổng môn học (Fuzzy): 102

=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE ===
Nhập 'exit' để thoát.

📘 Nhập nội dung muốn tra cứu: 4 sang chu

🔎 Kết quả tìm thấy (BẢNG):

### 📊 Kết quả Tra cứu Bảng (Độ khớp: 100%)

#### Bảng: Thang Diem 4
| Thang điểm chữ   | Thang điểm 4   |
|:-----------------|:---------------|
| A                | 4,0            |
| B+               | 3,5            |
| B                | 3,0            |
| C+               | 2,5            |
| C                | 2,0            |
| D+               | 1,5            |
| D                | 1,0            |
| F+               | 0,5            |
| F                | 0,0            |


-----------------------------

📘 Nhập nội dung muốn tra cứu: chu sang 4

🔎 Kết quả tìm thấy (BẢNG):

### 📊 Kết quả Tra cứu Bảng (Độ khớp: 95%)

#### Bảng: Thang Diem 4
| Thang điểm chữ   | Thang điểm 4   |
|:-----------------|:---------------|
| A                | 4,0            |
| B+

KeyboardInterrupt: Interrupted by user